# Detection of TOXicity in comments in Spanish (DETOXIS 2021)

## SESIÓN 2.1: Preprocesamiento y extracción de características

### Realizado por Álvaro Mazcuñán y Miquel Marín

#### Librerías

In [1]:
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

import re
import string
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Se lee el CSV con los datos de `DETOXIS`.

También se crean dos funciones claves. La primera, `read_data`, obtiene a partir de los datos del CSV, las variables que necesitaremos para el análisis.

La segunda de ellas, `tweet_preprocessing`, realiza el preprocesado correspondiente al primer apartado de esta tarea:
 - **Normalización**: Llevar todo el texto al mismo nivel
     - Conversión de todas las letras a mayúsculas o minúsculas
     - Tokenización
     - Eliminación de ruido: caracteres no deseados, como URLs, signos de puntuación, caracteres especiales, etc
     - Eliminación de palabras irrelevantes (stopwords)
 - **Stemming**: Obtener la raíz de una palabra
 - **Lematización**: Obtener la forma base (lema)

Su output **NO** está tokenizado. Esto es así debido a que 3 de las 4 funciones que se emplean más tardes requieren que el corpus no esté tokenizado. Cuando se utilice `Word2Vec` ya se modificará la función para tener los mensajes tokenizados.

#### Leer tweets y preprocesado 

In [2]:
data = pd.read_csv('data/DATASET_DETOXIS.csv')

def read_data(data):
    comment = list(data['comment'])
    toxicity = list(data['toxicity'])
    toxicity_level = list(data['toxicity_level'])
    return comment, toxicity, toxicity_level

def tweet_preprocessing_not_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return " ".join(lemma_tweets) # NO TOKENIZADO
    
comments, toxicity, toxicity_level = read_data(data)
tweets_cleaned = [tweet_preprocessing_not_tokenized(tweet) for tweet in comments]

In [3]:
tweets_cleaned[0]

'pensó zumo restar'

La lista `tweets_cleaned` contiene cada tweet ya preprocesado.

### Extracción de características

#### Bolsa de palabras (Bag-of-Words)

La primera de las diferentes opciones que se van a implementar es la creación de una Bolsa de Palabras o `Bag-of-Words`. En este caso, el valor por defecto del parámetro `ngram_range` es (1,1), queriendo decir que obtiene la bolsa de palabras con unigramas.

In [4]:
vectorizer = CountVectorizer()
vectorizer.fit(tweets_cleaned)

X_bag_of_words = vectorizer.transform(tweets_cleaned)

No se muestra el contenido de `X_bag_of_words` debido a que es una matriz de 3463 filas por 12700 columnas donde cada entrada representa las veces que aparece una palabra del vocabulario en el texto.

Si se desea conocer qué palabras se encuentran en el vocabulario, basta con ejecutar la siguiente línea: `vectorizer.get_feature_names()`

Aquí se muestra como queda con solo 10 palabras.

In [5]:
vectorizer.get_feature_names()[1000:1010]

['aprendemo',
 'aprenden',
 'aprendida',
 'aprendido',
 'aprendieran',
 'aprendái',
 'aprendí',
 'aprietan',
 'aprobado',
 'apropiación']

#### N-gramas de palabras. N-gramas de caracteres 

A continuación, se realizará la tarea por **N-gramas** debido a lo siguiente: Si creamos la lista de palabras mediante unigramas, entonces no se pueden capturar frases y expresiones de varias palabras, porque no tiene en cuenta el orden de las palabras.

No obstante, si se hace uso de n-gramas, por ejemplo, bigramas o trigramas, entonces se tendrán en cuenta las apariciones de pares o tripletas de palabras consecutivas, pudiendo realizar análisis más interesantes.


En este caso se realizará una bolsa de palabras utilizando **bigramas y trigramas**, que son N-gramas de 2 y 3 palabras respectivamente (pasando como parámetro `ngram_range=(2,3)`, ya que pensamos que, al tener un texto con gran cantidad de tweets, utilizando este enfoque se obtendrán mejores resultados cuando se realice la tarea de **Evaluación (2.3)** donde se entrenarán modelos como Regresión Logística, Máquina de Soporte Vectorial (SVM), árboles de decisión, etc.

Además, se ha investigado acerca del "valor óptimo" de ngramas y, en la mayoría de papers relacionados con NLP, se comenta que bigramas y trigramas son los que producen mejores resultados: 

<blockquote> <p>Payal B. Awachate, Prof. Vivek P. Kshirsagar <em>Improved Twitter Sentiment Analysis Using N
Gram Feature Selection and Combinations</em>. Computer Science and Engineering Department, Government College of Engineering, Aurangabad, India (September 2016)</p>
</blockquote>

Sin embargo, una vez nos encontremos en esa fase correspondiente y no se obtiene un buen accuracy, se podrá volver hacia atrás y cambiar algunos parámetros referentes a los N-gramas.

In [6]:
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 3))
counts = ngram_vectorizer.fit_transform(tweets_cleaned)
counts.toarray().astype(int)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### Term Frequency - Inverse Document Frequency (TF-IDF)

Consiste en una medida numérica que expresa cuán relevante es una palabra para un texto en un
corpus.

La idea con TF-IDF es reducir el peso de los términos proporcionalmente al número de textos en los
que aparecen. De esta forma, el valor de un término aumenta proporcionalmente al número de veces
que aparece en el texto, y es compensado por su frecuencia en el corpus.

Para realizar dicha tarea, basta con implementar las siguientes líneas de código:

In [7]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(tweets_cleaned)

TfidfVectorizer()

Si se desea la importancia de las palabras, basta con ejecutar la siguiente línea: `tfidf_vectorizer.vocabulary_`

Aquí se muestra para las 10 primeras.

In [8]:
iter_ = 0
for word in tfidf_vectorizer.vocabulary_.keys():
    if iter_ < 10:
        print(word, tfidf_vectorizer.vocabulary_[word])        
    else:
        break
    iter_ += 1

pensó 9003
zumo 12654
restar 10533
gusta 5821
afeitado 489
seco 10902
gent 5666
asi 1169
maten 7609
alta 714


#### Word embeddings - Word2Vec

Por último, se pasa a Words Embeddings, con `Word2Vec`.

Este modelo se encuentra disponible mediante dos formas:
 - CBOW: Continuous Bag-of-Words
 - Skip-Gram
 

<blockquote>
    Skip-Gram is more
efficient with small training data. Moreover, infrequent words are well presented. On the other hand, CBOW is faster and works well with frequent words. 
    
    <p>Marwa Naili, Anja Habacha Chaibi, Henda Hajjami Ben Ghezala <em>Comparative study of work embedding methods in topic segmentation</em>. </p>
    
    
</blockquote>

El modelo Word2Vec, de forma resumida, realiza lo siguiente: dado un vocabulario generado con las palabras del corpus, el objetivo es, para una palabra dada, que el modelo nos diga la probabilidad de que cada palabra del vocabulario sea vecina de la primera.

<img src="word2vec.png" width="800"/>

Según lo que se ha podido leer en este paper, se ha decidido realizar el modelo de Word2Vec mediante `Skip-Gram`, ya que la muestra de tweets que se están tratando en esta base de datos es de alrededor de los 3000, pudiendo considerarse un muestra pequeña.


Se modifica la función de antes para tener tokenizadas las palabras de cada tweet.

In [9]:
def tweet_preprocessing_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return lemma_tweets # TOKENIZADO
    
comments, toxicity, toxicity_level = read_data(data)
tweets_cleaned = [tweet_preprocessing_tokenized(tweet) for tweet in comments]

A continuación se pasa a crear el modelo de Word2Vec, añadiendo un parámetro `sg` donde se indica que se utilizará el enfoque de Skip-Gram `(sg=1)`

In [10]:
model = Word2Vec(
        tweets_cleaned,
        vector_size=30,
        min_count=5,
        sg=1)

Para mostrar un ejemplo del resultado obtenido utilizando dicho modelo, se probará **model** para que busque por las palabras de mayor similitud con **casa**.

In [11]:
model.wv.most_similar('casa')

[('dentro', 0.9911617040634155),
 ('venir', 0.9911611080169678),
 ('piso', 0.9903740286827087),
 ('visado', 0.9903460741043091),
 ('vinieron', 0.9898891448974609),
 ('2500', 0.989872395992279),
 ('derivar', 0.9898684620857239),
 ('viviendo', 0.98983234167099),
 ('pai', 0.9898146986961365),
 ('buscars', 0.989814043045044)]

In [13]:
def tweet_preprocessing_not_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return " ".join(lemma_tweets) # NO TOKENIZADO
    
comments, toxicity, toxicity_level = read_data(data)
tweets_cleaned = [tweet_preprocessing_not_tokenized(tweet) for tweet in comments]

model = Word2Vec(
        tweets_cleaned,
        vector_size=30,
        min_count=5,
        sg=1)

In [14]:
import pandas as pd
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec

In [18]:
data = pd.read_csv('data/DATASET_DETOXIS.csv')
sample_data = data[["comment", "toxicity","toxicity_level"]]



In [19]:
def tweet_preprocessing_not_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return " ".join(lemma_tweets) # NO TOKENIZADO

preprocessing = lambda x: tweet_preprocessing_not_tokenized(x)


sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))
train_X_, test_X_, train_Y_, test_Y_ = train_test_split(sample_data['comment'], sample_data['toxicity_level'], test_size=0.3)

<ipython-input-19-8d11717da6f6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))


In [44]:
model = Word2Vec(
        tweets_cleaned,
        vector_size=30,
        min_count=1,
        sg=1)

In [49]:
words = sorted(model.wv.vocab.key_to_index)
print("Number of words:", len(words))

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [41]:
model.build_vocab(sample_data['comment'])

In [43]:
model.train(sample_data['comment'])

ValueError: You must specify either total_examples or total_words, for proper learning-rate and progress calculations. If you've just built the vocabulary using the same corpus, using the count cached in the model is sufficient: total_examples=model.corpus_count.

In [ ]:
tfidf_vect_ = TfidfVectorizer()
tfidf_vect_.fit(sample_data['comment'])
train_X_Tfidf_ = tfidf_vect_.transform(train_X_)
test_X_Tfidf_ = tfidf_vect_.transform(test_X_)

In [61]:
train_X, test_X, train_Y, test_Y = train_test_split(sample_data['comment'], sample_data['toxicity'], test_size=0.3)
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))

ngram_vectorizer.fit(sample_data['comment'])
train_X_count_ = ngram_vectorizer.transform(train_X)
test_X_count_ = ngram_vectorizer.transform(test_X)

svm_clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm_clf.fit(train_X_count_,train_Y)
predictions_SVM = svm_clf.predict(test_X_count_)

score_svm = f1_score(test_Y, predictions_SVM, average='macro')
score_svm

In [62]:
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))

ngram_vectorizer.fit(sample_data['comment'])
train_X_count_ = ngram_vectorizer.transform(train_X)
test_X_count_ = ngram_vectorizer.transform(test_X)

In [63]:
svm_clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm_clf.fit(train_X_count_,train_Y)
predictions_SVM = svm_clf.predict(test_X_count_)

In [64]:
score_svm = f1_score(test_Y, predictions_SVM, average='macro')
score_svm

0.6959421399589857